In [19]:
#Importing modules
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import requests
from scrapy.http import HtmlResponse
import pandas as pd
import pyodbc

## Extracting data from the Capetown fresh produce market

In [2]:
#Enabling firefox driver
driver = webdriver.Firefox()
driver.get('https://www.ctmarket.co.za/daily-prices/')

In [3]:
#Parse the html content
soup = BeautifulSoup(driver.page_source, "html.parser")

In [4]:
#finding tables
my_table = soup.find('table',id="table_1")

In [5]:
#Extracting the column names
columns=my_table.find_all('th')
columns_names=[col.text for col in columns]
columns_names

['item',
 'description',
 'grade',
 'container',
 'container_count',
 'mass',
 'low_price',
 'high_price',
 'avg_price',
 'date_updated']

In [6]:
#Extracting the rows
rows = [x.find_all('td') for x in  my_table.find_all('tr')][1:-1]

for item in range(len(rows)):
    for val in range(len(rows[0])):
        rows[item][val]=rows[item][val].text
    
rows[0]

['APER',
 'APPLE EARLY RED ONE',
 '1X',
 'MARK 4 (18.3kg)',
 '80',
 '18.30',
 '150.00',
 '150.00',
 '150.00',
 '25/08/2020 11:44 AM']

In [7]:
#Saving the data into a DataFrame
Capetown_df=pd.DataFrame(rows)
Capetown_df.columns=columns_names
Capetown_df

,item,description,grade,container,container_count,mass,low_price,high_price,avg_price,date_updated
0,APER,APPLE EARLY RED ONE,1X,MARK 4 (18.3kg),80,18.30,150.00,150.00,150.00,25/08/2020 11:44 AM
1,APGA,APPLE GALA,2M,JUMBLE BOX (7KG),,7.00,35.00,45.00,40.67,25/08/2020 11:44 AM
2,APGA,APPLE GALA,1L,MARK 4 (18.3kg),100,18.30,160.00,160.00,160.00,25/08/2020 11:44 AM
3,APGD,APPLE GOLDEN DELICIOUS,2M,BAG/PACKET (1KG),,1.00,10.00,10.00,10.00,25/08/2020 11:44 AM
4,APGD,APPLE GOLDEN DELICIOUS,1M,BAG/PACKET,,1.50,11.00,11.00,11.00,25/08/2020 11:44 AM
...,...,...,...,...,...,...,...,...,...,...
545,TOSE,TOMATOES SALADETTE,1M,PREPACK BOX (7.5KG),,7.50,85.00,150.00,145.36,25/08/2020 11:44 AM
546,TURN,Turnips,,Decon (8kg),,8.00,40.00,80.00,52.38,25/08/2020 11:44 AM
547,VGMS,Vegetables Miscellaneous,,Packet (500g),,0.50,40.00,40.00,40.00,25/08/2020 11:44 AM
548,WBLM,Waterblommetjies,,Punnet - each (500g),,0.50,36.00,36.00,36.00,25/08/2020 11:44 AM


## Extracting data from the Durban fresh produce market

In [8]:
#Creating a webdriver
driver2 = webdriver.Firefox()
driver2.get('http://www.durban.gov.za:152/')

In [9]:
#Parse the html content
soup2 = BeautifulSoup(driver2.page_source, "html.parser")

In [10]:
#Column Names
table=soup2.find('table')
columns=[col.text.strip() for col in table.find_all('th')]
columns[0]='Commodities'
columns

['Commodities',
 'Weight (Kg)',
 'Size Grade',
 'Container',
 'Province',
 'Low Price',
 'High Price',
 'Average Price',
 'Sales Total',
 'Total Qty Sold',
 'Total Kg Sold',
 'Stock On Hand',
 'Date']

In [11]:
#Getting all the items
rows=table.find_all('tr')[1:]
items=[[row.text.strip() for row in item.find_all('td')] for item in rows]
items[0]

['ABC JUICE',
 '0.33',
 '',
 'OA003',
 'TRANSVAAL',
 '6.52',
 '6.52',
 '6.52',
 '130.4',
 '20',
 '6.6',
 '1440',
 '25/Aug/2020']

In [12]:
#Saving the results as a dataframe
Durban_df=pd.DataFrame(items)
Durban_df.columns=columns
Durban_df

,Commodities,Weight (Kg),Size Grade,Container,Province,Low Price,High Price,Average Price,Sales Total,Total Qty Sold,Total Kg Sold,Stock On Hand,Date
0,ABC JUICE,0.33,,OA003,TRANSVAAL,6.52,6.52,6.52,130.4,20,6.6,1440,25/Aug/2020
1,ABC JUICE,0.75,,OB008,TRANSVAAL,14.70,14.70,14.70,147,10,7.5,510,25/Aug/2020
2,AMADUMBE,13.50,,A1135,NATAL,150.00,160.00,151.11,13600,90,1215,10,25/Aug/2020
3,AMADUMBE,25.00,,SC250,NATAL,350.00,350.00,350.00,350,1,25,0,25/Aug/2020
4,APPLE,0.33,,OA003,TRANSVAAL,6.52,6.52,6.52,195.6,30,9.9,2130,25/Aug/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,TURNIPS,0.50,,NE005,NATAL,0.00,0.00,0.00,0,0,0,15,25/Aug/2020
847,VEGETABLES MISCELLANEOUS,1.00,,EA010,NATAL,20.00,180.00,81.06,2675,33,33,194,25/Aug/2020
848,WATERCREST,6.00,,BE060,NATAL,80.00,80.00,80.00,400,5,30,7,25/Aug/2020
849,ZOBO BEETROOT INFUSED JUICE,0.33,,OA003,TRANSVAAL,6.52,6.52,6.52,130.4,20,6.6,2380,25/Aug/2020


## Extracting data from the Joburg fresh produce market

In [20]:
# Url where the data is going to be scrapped from
url = 'http://www.joburgmarket.co.za/dailyprices.php'

In [21]:
# Extracting html content of the url
html = requests.get(url).content
# Creating a response variable to enable web scrapping in the url 
response = HtmlResponse(url=url, body=html)

### scrapping_date 

In [22]:
# Extracting the date for latest information displayed on the website
date = response.xpath('//div[@id="right2"]').css('p b ::text').get()
print(f"The latest information in the Joburg market website is for {date}.")

The latest information in the Joburg market website is for 25 August 2020.


In [25]:
date_df = pd.DataFrame([date], columns=['date'])

In [26]:
date_df

,date
0,25 August 2020


### commodity_raw update

In [27]:
commodity_df = pd.read_html(url)[0]

In [28]:
commodity_df.head()

,Commodity,Total Value Sold,Total Qty Sold,Total Kg Sold,Qty Available
0,AMADUMBE,"R420.00MTD: R3,820.00",7MTD: 75,14MTD: 150,0
1,APPLES,"R1,996,281.00MTD: R28,315,967.00","22,996MTD: 324,641","274,778MTD: 3,941,434",92879
2,ARTICHOKES,"R3,480.00MTD: R59,700.00",27MTD: 492,30MTD: 582,1
3,ASPARAGUS,"R17,800.00MTD: R77,570.00",61MTD: 316,153MTD: 790,1
4,ATCHARA,"R72.00MTD: R1,501.20",1MTD: 25,4MTD: 73,205


In [29]:
# Attach the date column to the dataframe
commodity_df.insert(loc=0, column='Date', value=date)

In [30]:
commodity_df.head()

,Date,Commodity,Total Value Sold,Total Qty Sold,Total Kg Sold,Qty Available
0,25 August 2020,AMADUMBE,"R420.00MTD: R3,820.00",7MTD: 75,14MTD: 150,0
1,25 August 2020,APPLES,"R1,996,281.00MTD: R28,315,967.00","22,996MTD: 324,641","274,778MTD: 3,941,434",92879
2,25 August 2020,ARTICHOKES,"R3,480.00MTD: R59,700.00",27MTD: 492,30MTD: 582,1
3,25 August 2020,ASPARAGUS,"R17,800.00MTD: R77,570.00",61MTD: 316,153MTD: 790,1
4,25 August 2020,ATCHARA,"R72.00MTD: R1,501.20",1MTD: 25,4MTD: 73,205


### container_raw update

In [31]:
# Extract value that links it to the commodity website for a detailed stats
commodity_values = response.xpath('//select/option/@value').extract()[1:] # Exclude 'All' option
# Extract commodity name
commodity_names = response.xpath('//table[@class="alltable"]').css('td.tleft2 ::text').extract()

In [32]:
for index, (name, value) in enumerate(zip(commodity_names, commodity_values)):
    # URL for container information which will update value for each commodity
    url_one = f'http://www.joburgmarket.co.za/dailyprices.php?commodity={value}&containerall=1'
    
    # Initially create a DataFrame 
    if not index:
        
        container_df = pd.read_html(url_one)[0]
        # Attach the commodity name column to the container dataframe
        container_df.insert(loc=0, column='commodity', value=name)
        
    else:
        
        # Create a temporary DataFrame
        temp_df = pd.read_html(url_one)[0]
        # Attach the commodity name column to the temporary dataframe
        temp_df.insert(loc=0, column='commodity', value=name)
        # Concatenate temporary DataFrame to the container DataFrame
        container_df = pd.concat([container_df, temp_df], ignore_index=True)

In [33]:
# Attach the date column to the dataframe
container_df.insert(loc=0, column='Date', value=date)

In [34]:
container_df.head()

,Date,commodity,Container,Qty Available,Value Sold,Qty Sold,Kg Sold,Average Price per Kg
0,25 August 2020,AMADUMBE,2KG BOX,0.0,"R420.00MTD: R3,820.00",7MTD: 75,14MTD: 150,R30
1,25 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,16.0,"R0.00MTD: R42,032.00",0MTD: 548,"0MTD: 5,480",R0
2,25 August 2020,APPLES,11KG JUMBLE CARTON,805.0,"R19,350.00MTD: R292,294.00","291MTD: 4,258","3201MTD: 46,838",R6.04
3,25 August 2020,APPLES,12 X 1KG ECONO PACK CARTON,1424.0,"R59,060.00MTD: R1,808,040.00","664MTD: 21,594","7968MTD: 259,128",R7.41
4,25 August 2020,APPLES,12.5KG M6 CARTON,11.0,"R0.00MTD: R17,003.00",0MTD: 287,"0MTD: 3,588",R0


### product_combination_raw update

In [35]:
for index, (name, value) in enumerate(zip(commodity_names, commodity_values)):
    # URL for container information which will update value for each commodity
    url_two = f'http://www.joburgmarket.co.za/dailyprices.php?commodity={value}&containerall=2'
    
    # Initially create a DataFrame 
    if not index:
        
        product_combo_df = pd.read_html(url_two)[0]
        # Attach the commodity name column to the product_combo dataframe
        product_combo_df.insert(loc=0, column='commodity', value=name)
        
    else:
        
        # Create a temporary DataFrame
        temp_df = pd.read_html(url_two)[0]
        # Attach the commodity name column to the temporary dataframe
        temp_df.insert(loc=0, column='commodity', value=name)
        # Concatenate temporary DataFrame to the product_combo DataFrame
        product_combo_df = pd.concat([product_combo_df, temp_df], ignore_index=True)

In [36]:
product_combo_df.tail()

,commodity,Container,Unit Mass,Product Combination,Total Value Sold,Total Qty Sold,Total Kg Sold,Average,Highest Price,Ave per Kg,Highest Price per Kg
2109,YELLOW PEPPERS,5KG BOX,5.0,"*,CL 2,L,*,*","R1,350.00",90.0,450.0,R15.00,R15.00,R3.00,R3.00
2110,YELLOW PEPPERS,5KG BOX,5.0,"*,CL 2,M,*,*","R4,260.00",81.0,405.0,R52.59,R60.00,R10.52,R12.00
2111,YELLOW PEPPERS,5KG BOX,5.0,"*,CL 2,S,*,*",R0.00,0.0,0.0,R0.00,R0.00,R0.00,R0.00
2112,YELLOW PEPPERS,5KG POCKET,5.0,"*,*,*,*,*",R0.00,0.0,0.0,R0.00,R0.00,R0.00,R0.00
2113,YELLOW PEPPERS,8KG BOX,8.0,"*,*,*,*,*",R0.00,0.0,0.0,R0.00,R0.00,R0.00,R0.00


In [37]:
# Attach the date column to the dataframe
product_combo_df.insert(loc=0, column='Date', value=date)

In [38]:
product_combo_df.head()

,Date,commodity,Container,Unit Mass,Product Combination,Total Value Sold,Total Qty Sold,Total Kg Sold,Average,Highest Price,Ave per Kg,Highest Price per Kg
0,25 August 2020,AMADUMBE,2KG BOX,2.0,"*,*,*,*,*",R420.00,7.0,14.0,R60.00,R60.00,R30.00,R30.00
1,25 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,10.0,"GOLDEN DELICIOUS,CL 1,*,*,*",R0.00,0.0,0.0,R0.00,R0.00,R0.00,R0.00
2,25 August 2020,APPLES,11KG JUMBLE CARTON,11.0,"CRIPPS PINK,CL 2,S,*,*","R5,580.00",90.0,990.0,R62.00,R62.00,R5.64,R5.64
3,25 August 2020,APPLES,11KG JUMBLE CARTON,11.0,"FUJI,CL 2,L,*,*","R6,610.00",94.0,1034.0,R70.32,R80.00,R6.39,R7.27
4,25 August 2020,APPLES,11KG JUMBLE CARTON,11.0,"FUJI,CL 2,M,*,*",R180.00,3.0,33.0,R60.00,R60.00,R5.45,R5.45


## Saving the data into an SQL Database

In [39]:
#Creating a connection
conn = pyodbc.connect(driver='{SQL Server}',
                      host='DESKTOP-0SIPQ7F\SQLEXPRESS',
                      database='Fresh_Produce_Market_Data', #Database name
                      trusted_connection='tcon',
                      user='sa')

In [40]:
# Return a new Cursor object using the connection.
cur = conn.cursor()

### Inserting Data into a table for Capetown Fresh Produce

In [15]:
#Inserting data into an sql database
for index,row in Capetown_df.iterrows():
    cur.execute("INSERT INTO Capetown_Fresh_produce_market(item, description, grade,container,container_count,mass,low_price,high_price,avg_price,date_updated) Values(?,?,?,?,?,?,?,?,?,?)",
                row['item'],
                row['description'],
                row['grade'],
                row['container'],
                row['container_count'],
                row['mass'],
                row['low_price'],
                row['high_price'],
                row['avg_price'],
                row['date_updated'])
    
    conn.commit()

### Inserting Data into a table for Durban Fresh Produce

In [16]:
#Inserting data into an sql database
for index,row in Durban_df.iterrows():
    cur.execute("INSERT INTO Durban_Fresh_produce_market(Commodities, Weight_Kg, Size_Grade,Container,Province,Low_Price,High_Price,Average_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,Date) Values(?,?,?,?,?,?,?,?,?,?,?,?,?)",
                row['Commodities'],
                row['Weight (Kg)'],
                row['Size Grade'],
                row['Container'],
                row['Province'],
                row['Low Price'],
                row['High Price'],
                row['Average Price'],
                row['Sales Total'],
                row['Total Qty Sold'],
                row['Total Kg Sold'],
                row['Stock On Hand'],
                row['Date'])
    
    conn.commit()

### Inserting Data into a table for Joburg_Fresh_produce_scrapping_date

In [42]:
#Inserting data into an sql database
for index,row in date_df.iterrows():
    cur.execute("INSERT INTO Joburg_Fresh_produce_scrapping_date(date) Values(?)",
                row['date'])
    
    conn.commit()

### Inserting Data into a table for Joburg_Fresh_produce_commodity_raw

In [51]:
for index,row in commodity_df.iterrows():
    cur.execute("INSERT INTO Joburg_Fresh_produce_commodity_raw(date, commodity, total_value_sold,total_qty_sold,total_kg_sold,qty_available) Values(?,?,?,?,?,?)",
                row['Date'],
                row['Commodity'],
                row['Total Value Sold'],
                row['Total Qty Sold'],
                row['Total Kg Sold'],
                row['Qty Available'])
    
    conn.commit()

### Inserting Data into a table for Joburg_Fresh_produce_container_raw

In [52]:
container_df.head()

,Date,commodity,Container,Qty Available,Value Sold,Qty Sold,Kg Sold,Average Price per Kg
0,25 August 2020,AMADUMBE,2KG BOX,0.0,"R420.00MTD: R3,820.00",7MTD: 75,14MTD: 150,R30
1,25 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,16.0,"R0.00MTD: R42,032.00",0MTD: 548,"0MTD: 5,480",R0
2,25 August 2020,APPLES,11KG JUMBLE CARTON,805.0,"R19,350.00MTD: R292,294.00","291MTD: 4,258","3201MTD: 46,838",R6.04
3,25 August 2020,APPLES,12 X 1KG ECONO PACK CARTON,1424.0,"R59,060.00MTD: R1,808,040.00","664MTD: 21,594","7968MTD: 259,128",R7.41
4,25 August 2020,APPLES,12.5KG M6 CARTON,11.0,"R0.00MTD: R17,003.00",0MTD: 287,"0MTD: 3,588",R0


In [53]:
for index,row in container_df.iterrows():
    cur.execute("INSERT INTO Joburg_Fresh_produce_container_raw(date, commodity,container, qty_available,value_sold,qty_sold,kg_sold,average_price_per_kg) Values(?,?,?,?,?,?,?,?)",
                row['Date'],
                row['commodity'],
                row['Container'],
                row['Qty Available'],
                row['Value Sold'],
                row['Qty Sold'],
                row['Kg Sold'],
                row['Average Price per Kg'])
    
    conn.commit()

In [54]:
product_combo_df.head()

,Date,commodity,Container,Unit Mass,Product Combination,Total Value Sold,Total Qty Sold,Total Kg Sold,Average,Highest Price,Ave per Kg,Highest Price per Kg
0,25 August 2020,AMADUMBE,2KG BOX,2.0,"*,*,*,*,*",R420.00,7.0,14.0,R60.00,R60.00,R30.00,R30.00
1,25 August 2020,APPLES,10 X 1KG ECONO PACK CARTON,10.0,"GOLDEN DELICIOUS,CL 1,*,*,*",R0.00,0.0,0.0,R0.00,R0.00,R0.00,R0.00
2,25 August 2020,APPLES,11KG JUMBLE CARTON,11.0,"CRIPPS PINK,CL 2,S,*,*","R5,580.00",90.0,990.0,R62.00,R62.00,R5.64,R5.64
3,25 August 2020,APPLES,11KG JUMBLE CARTON,11.0,"FUJI,CL 2,L,*,*","R6,610.00",94.0,1034.0,R70.32,R80.00,R6.39,R7.27
4,25 August 2020,APPLES,11KG JUMBLE CARTON,11.0,"FUJI,CL 2,M,*,*",R180.00,3.0,33.0,R60.00,R60.00,R5.45,R5.45


In [55]:
for index,row in product_combo_df.iterrows():
    cur.execute("INSERT INTO Joburg_Fresh_produce_product_combination_raw(date, commodity,container, unit_mass,product_combination,total_value_sold,total_qty_sold,total_kg_sold,average,highest_price,ave_per_kg,highest_price_per_kg) Values(?,?,?,?,?,?,?,?,?,?,?,?)",
                row['Date'],
                row['commodity'],
                row['Container'],
                row['Unit Mass'],
                row['Product Combination'],
                row['Total Value Sold'],
                row['Total Qty Sold'],
                row['Total Kg Sold'],
                row['Average'],
                row['Highest Price'],
                row['Ave per Kg'],
                row['Highest Price per Kg'])
    
    conn.commit()

## Closing All connections

In [56]:
driver.close()
driver2.close()
cur.close()  
conn.close()